# LecoSpec Data Munging

In [1]:
# notebooks use their location as their working directory, so
# if we are in a subfolder, move to the main folder.  
# This however can safely be run multiple times
#setwd(M:/lecospec/lecospec)
if(!dir.exists("Functions/")){
    setwd("../../")
    if(!dir.exists("Functions")){
        setwd("M:/lecospec/lecospec/")
    }
}
source("Functions/lecospectR.R", echo = FALSE)

Loading required package: tidyverse

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.3.6      ✔ purrr   0.3.5 
✔ tibble  3.1.8      ✔ dplyr   1.0.10
✔ tidyr   1.2.1      ✔ stringr 1.4.1 
✔ readr   2.1.3      ✔ forcats 0.5.2 
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
Loading required package: compiler

Loading required package: raster

Loading required package: sp


Attaching package: ‘raster’


The following object is masked from ‘package:dplyr’:

    select


Loading required package: hsdar

Loading required package: rgdal

Please note that rgdal will be retired by the end of 2023,
plan transition to sf/stars/terra functions using GDAL and PROJ
at your earliest convenience.

rgdal: version: 1.5-32, (SVN revision 1176)
Geospatial Data Abstraction Library extensions to R successfully loaded
Loaded GDAL runtime: GDAL 3.4.1, relea

## Notation

Throughout the notebook, variables starting with `img_` are UAV image-based information (data, filepaths, etc).  Similarly, variables beginning with `grd_` related to data collected on the ground.  

Also, some other naming conventions for variables with data transformations:
* `robust` in a variable name refers to data treated by center according to the median and scaling by teh inter-quartile range (a la sklearns RobustScaler)
* `minmax` (and its ilk) are min-max scaled data, i.e. scaled to the interval [0,1] by subtracting the minimum and dividing by the range.
* `standard(ized)` refers to data treated with with the z-score transform by centring using the mean and scaling y the standard deviation (like sklearns StandardScaler)
* `corrected` means that a linear transformation has been applied to account for differences in sensor calibration.
* `raw` refers to having no transformations applied
* `clipped` means that outliers have been clipped to the upper and lower fence values based on the Inter-Quartile Range method. 
* `imputed` means that outliers have been removed and imputed
* `dropped` means that dataframe rows containing outliers have been removed

Example: `img_robust_indices` refers to vegetation indices from the UAV images treated with the robust scaler. 

## Define data locations


In [2]:
# spectral library
grd_base_path <- "./Output/C_001_SC3_Cleaned_SpectralLib.csv"
grd_speclib <- read.csv(grd_base_path, header = TRUE)
#grd_index_path <- ./Data/D_002_SpecLib_Derivs.csv
#grd_indices <- read.csv(grd_index_path)
# this data has some lines that have no labels, so we remove them 
grd_speclib <- grd_speclib[!is.na(grd_speclib$Functional_group1),]
head(grd_speclib)

,X,ScanID,Area,Code_name,Species_name,Functional_group1,Functional_group2,Species_name_Freq,Functional_group1_Freq,Functional_group2_Freq,⋯,Radiometric.Calibration,Units,Latitude,Longitude,Altitude,GPS.Time,Satellites,Calibrated.Reference.Correction.File,Channels,ScanNum
,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<int>,<int>,⋯,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<int>,<int>
1,1,aleoch_Murph_061,Murphy,aleoch,Alectoria ochroleuca,Lichen,LightTerrestrialMacrolichen,6,453,118,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
2,2,aleoch_Murph_063,Murphy,aleoch,Alectoria ochroleuca,Lichen,LightTerrestrialMacrolichen,6,453,118,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
3,3,aleoch_Murph_064,Murphy,aleoch,Alectoria ochroleuca,Lichen,LightTerrestrialMacrolichen,6,453,118,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
4,4,aleoch_Murph_065,Murphy,aleoch,Alectoria ochroleuca,Lichen,LightTerrestrialMacrolichen,6,453,118,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
5,5,aleoch_Murph_066,Murphy,aleoch,Alectoria ochroleuca,Lichen,LightTerrestrialMacrolichen,6,453,118,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
6,6,alnfru_00003,Yukon_Delta,alnfru,Alnus sp.,ShrubDecid,ShrubAlder,82,360,82,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


In [3]:
img_base_path <- "Data/Ground_Validation/PFT_image_spectra/PFT_Image_SpectralLib_Clean.csv"
img_speclib <- read.csv(img_base_path)

# currently, not using the old pre-proccessing scheme and just doing it here.
#img_index_path <- Data/D_002_Image_SpecLib_Derivs.csv
#img_speclib <- read.csv(img_base_path)
head(img_speclib)

,X,UID,ScanNum,sample_name,PFT,FncGrp1,Site,X398,X399,X400,⋯,X990,X991,X992,X993,X994,X995,X996,X997,X998,X999
,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1,BisonGulchPFTsBetula1,1,spec_1,Betula,TreeBroadleaf,BisonGulch,0.05814769,0.05926529,0.06028869,⋯,0.6815182,0.6811660,0.6890470,0.7040298,0.7249807,0.7507566,0.7801884,0.8121027,0.8453261,0.8786852
2,2,BisonGulchPFTsBetula1,1,spec_2,Betula,TreeBroadleaf,BisonGulch,0.04456014,0.04778814,0.05079318,⋯,0.6706666,0.6683159,0.6786394,0.7000307,0.7308801,0.7695067,0.8140391,0.8625739,0.9132079,0.9640378
3,3,BisonGulchPFTsBetula1,1,spec_3,Betula,TreeBroadleaf,BisonGulch,0.03929324,0.04265593,0.04557066,⋯,0.5152525,0.5091915,0.5178217,0.5395294,0.5726982,0.6156166,0.6663192,0.7227978,0.7830447,0.8450520
4,4,BisonGulchPFTsBetula1,1,spec_4,Betula,TreeBroadleaf,BisonGulch,0.13230228,0.11122692,0.09129034,⋯,0.5120581,0.5113880,0.5348292,0.5745538,0.6227243,0.6723311,0.7185860,0.7570701,0.7833644,0.7930498
5,5,BisonGulchPFTsBetula1,1,spec_5,Betula,TreeBroadleaf,BisonGulch,0.05211388,0.05565497,0.05878525,⋯,0.6863419,0.6680365,0.6509006,0.6344450,0.6181806,0.6017555,0.5851848,0.5685449,0.5519121,0.5353626
6,6,BisonGulchPFTsBetula1,1,spec_6,Betula,TreeBroadleaf,BisonGulch,0.06955397,0.06788242,0.06631141,⋯,0.7354495,0.7371508,0.7445194,0.7567953,0.7732173,0.7930235,0.8154512,0.8397375,0.8651196,0.8908347


Okay, there are some metadata columns that should not be there for the next step - lets remove them with `subset`

In [4]:
RawUID<- img_speclib %>% 
  dplyr::select(UID) %>% as.data.frame() #%>%

SiteNames<-str_split(RawUID[,1], "PFT") %>% 
  as.data.frame() %>% 
  t %>% 
  as.data.frame() %>%
  dplyr::rename(Site = V1) %>% 
  dplyr::select(Site)
print(SiteNames)

                                                     Site
c..BisonGulch....sBetula1..                    BisonGulch
c..BisonGulch....sBetula1...1                  BisonGulch
c..BisonGulch....sBetula1...2                  BisonGulch
c..BisonGulch....sBetula1...3                  BisonGulch
c..BisonGulch....sBetula1...4                  BisonGulch
c..BisonGulch....sBetula1...5                  BisonGulch
c..BisonGulch....sBetula1...6                  BisonGulch
c..BisonGulch....sBetula1...7                  BisonGulch
c..BisonGulch....sBetula1...8                  BisonGulch
c..BisonGulch....sBetula1...9                  BisonGulch
c..BisonGulch....sBetula1...10                 BisonGulch
c..BisonGulch....sBetula1...11                 BisonGulch
c..BisonGulch....sBetula1...12                 BisonGulch
c..BisonGulch....sBetula1...13                 BisonGulch
c..BisonGulch....sBetula1...14                 BisonGulch
c..BisonGulch....sBetula1...15                 BisonGulch
c..BisonGulch.

In [5]:
bg_speclib <- img_speclib[img_speclib$Site == "BisonGulch",]

In [6]:
img_bands <- subset(
    img_speclib, 
    select=-c(
        X,
    	UID,
        ScanNum,
    	sample_name,
    	PFT,
    	FncGrp1,
        Site
    ))


grd_bands <- subset(
    grd_speclib, 
    select=-c(
        X,
        ScanID,
        Area,
        Code_name,
        Species_name,
        Functional_group1,
        Functional_group2,
        Species_name_Freq,
        Functional_group1_Freq,
        Functional_group2_Freq,
        Genus,
        Version,
        File.Name,
        Instrument,
        Detectors,
        Measurement,
        Date,
        Time,
        Battery.Voltage,
        Averages,
        Integration1,
        Integration2,
        Integration3,
        Dark.Mode,
        Foreoptic,
        Radiometric.Calibration,
        Units,
        Latitude,
        Longitude,
        Altitude,
        GPS.Time,
        Satellites,
        Calibrated.Reference.Correction.File,
        Channels,
        ScanNum
    )
)

bg_bands <- subset(
    bg_speclib, 
    select=-c(
        X,
    	UID,
        ScanNum,
    	sample_name,
    	PFT,
    	FncGrp1,
        Site
    ))


In [7]:
bg_bands <- subset(
    bg_speclib, 
    select=-c(
        X,
    	UID,
        ScanNum,
    	sample_name,
    	PFT,
    	FncGrp1,
        Site
    ))

Calculate the vegetation indices from the spectral libraries - its easy with lecospectR!

Note that the image-based scpectra are normalized from zero to one, and the ground specctra are on the range zero to one hundred.  

In [8]:
img_indices <- get_vegetation_indices(img_bands, NULL)# should have a default of NULL, you know?
grd_indices <- get_vegetation_indices(grd_bands, NULL)
bg_indices <- get_vegetation_indices(bg_speclib, NULL)

In [9]:
write.csv(img_indices, file="Data/gs/x_train/img_indices_only.csv")

write.csv(grd_indices, file="Data/gs/x_train/grd_indices_only.csv")

write.csv(bg_indices, file = "Data/gs/x_train/bison_gulch_indices")

In [10]:
head(img_indices)
head(img_indices)

,Boochs,Boochs2,CARI,Carter,Carter2,Carter3,Carter4,Carter5,Carter6,CI,⋯,TCARI,TCARIOSAVI,TCARI2,TCARI2OSAVI2,TGI,TVI,Vogelmann,Vogelmann2,Vogelmann3,Vogelmann4
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,0.01118062,0.01252303,0.5520621,2.540064,0.2065954,0.1994095,0.4599336,1.640920,0.1548045,1.212547,⋯,0.2615574,0.3456065,0.4178572,0.9235753,8.286956,35.80558,1.439152,-0.08239895,1.064248,-0.09009970
2,0.01111790,0.01355575,0.5766599,2.625150,0.2065916,0.2070232,0.4513357,1.656177,0.1700914,1.175201,⋯,0.2831688,0.3720026,0.4496971,0.9654107,9.286506,38.17785,1.446272,-0.08680580,1.135065,-0.09518428
3,0.01084760,0.01378545,0.5614974,3.251237,0.1987500,0.2026890,0.4400645,1.585012,0.1803148,1.173642,⋯,0.2865432,0.3797346,0.4659473,0.9777083,10.063824,38.92187,1.454211,-0.07940521,1.153111,-0.08775410
4,0.01181043,0.01221932,0.5747244,2.602032,0.1809264,0.1767072,0.4379739,1.650827,0.1572204,1.084016,⋯,0.2773505,0.3524937,0.4235281,0.8672684,8.906298,40.04949,1.449666,-0.10812234,1.245379,-0.11775728
5,0.01119492,0.01259771,0.4934165,2.215177,0.1763305,0.1782588,0.4198506,1.671405,0.1517899,1.214389,⋯,0.2530670,0.3235553,0.3943958,0.7973731,8.621081,37.23711,1.486982,-0.09404361,1.060612,-0.10349363
6,0.01124955,0.01239871,0.5616888,2.578908,0.2044972,0.1928170,0.4591589,1.658333,0.1515471,1.234711,⋯,0.2609731,0.3423476,0.4145336,0.9106479,8.084553,36.27321,1.438798,-0.08098956,1.066427,-0.08849283


,Boochs,Boochs2,CARI,Carter,Carter2,Carter3,Carter4,Carter5,Carter6,CI,⋯,TCARI,TCARIOSAVI,TCARI2,TCARI2OSAVI2,TGI,TVI,Vogelmann,Vogelmann2,Vogelmann3,Vogelmann4
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,0.01118062,0.01252303,0.5520621,2.540064,0.2065954,0.1994095,0.4599336,1.640920,0.1548045,1.212547,⋯,0.2615574,0.3456065,0.4178572,0.9235753,8.286956,35.80558,1.439152,-0.08239895,1.064248,-0.09009970
2,0.01111790,0.01355575,0.5766599,2.625150,0.2065916,0.2070232,0.4513357,1.656177,0.1700914,1.175201,⋯,0.2831688,0.3720026,0.4496971,0.9654107,9.286506,38.17785,1.446272,-0.08680580,1.135065,-0.09518428
3,0.01084760,0.01378545,0.5614974,3.251237,0.1987500,0.2026890,0.4400645,1.585012,0.1803148,1.173642,⋯,0.2865432,0.3797346,0.4659473,0.9777083,10.063824,38.92187,1.454211,-0.07940521,1.153111,-0.08775410
4,0.01181043,0.01221932,0.5747244,2.602032,0.1809264,0.1767072,0.4379739,1.650827,0.1572204,1.084016,⋯,0.2773505,0.3524937,0.4235281,0.8672684,8.906298,40.04949,1.449666,-0.10812234,1.245379,-0.11775728
5,0.01119492,0.01259771,0.4934165,2.215177,0.1763305,0.1782588,0.4198506,1.671405,0.1517899,1.214389,⋯,0.2530670,0.3235553,0.3943958,0.7973731,8.621081,37.23711,1.486982,-0.09404361,1.060612,-0.10349363
6,0.01124955,0.01239871,0.5616888,2.578908,0.2044972,0.1928170,0.4591589,1.658333,0.1515471,1.234711,⋯,0.2609731,0.3423476,0.4145336,0.9106479,8.084553,36.27321,1.438798,-0.08098956,1.066427,-0.08849283


This is actually enough to start training models.  We have the vegetation indices, but instead of doing that, let's transform the data and write it to file.  Then we will proceed to creating the model corrections, etc.

In [11]:
img_resampled_bands <- resample_df(img_bands, drop_existing=TRUE)# corrects scale difference (poorly)
grd_resampled_bands <- resample_df(0.01*grd_bands, drop_existing=TRUE)
bg_resampled_bands <- resample_df(bg_bands, drop_existing=TRUE)# corrects scale difference (poorly)

head(img_resampled_bands)
head(grd_resampled_bands)

Using spline to predict value at new bands...

Beware the spectra are now partially smoothed.

Using spline to predict value at new bands...

Beware the spectra are now partially smoothed.

Using spline to predict value at new bands...

Beware the spectra are now partially smoothed.



,X402.593_5nm,X407.593_5nm,X412.593_5nm,X417.593_5nm,X422.593_5nm,X427.593_5nm,X432.593_5nm,X437.593_5nm,X442.593_5nm,X447.593_5nm,⋯,X947.593_5nm,X952.593_5nm,X957.593_5nm,X962.593_5nm,X967.593_5nm,X972.593_5nm,X977.593_5nm,X982.593_5nm,X987.593_5nm,X992.593_5nm
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,0.06212404,0.06180653,0.05693654,0.05199935,0.04943319,0.04810434,0.04814491,0.04839997,0.04902924,0.05122705,⋯,0.7765625,0.6580751,0.5365841,0.5196822,0.6809964,0.8697628,0.9139739,0.8369247,0.7141234,0.6972216
2,0.05749476,0.06297455,0.05918532,0.05290789,0.05137537,0.05212780,0.05244476,0.05221252,0.05233717,0.05398912,⋯,0.7950162,0.6789633,0.6053080,0.6241000,0.7584388,0.9404683,1.0002115,0.9083915,0.7245195,0.6902076
3,0.04999572,0.04854917,0.04249125,0.03980269,0.04341043,0.04719443,0.04936688,0.05086627,0.05222365,0.05477898,⋯,0.7504691,0.7452264,0.6078203,0.5917521,0.8167859,0.9920923,1.0107302,0.8651387,0.5893411,0.5292330
4,0.05353346,0.04877967,0.05867343,0.04561053,0.04907692,0.04761848,0.04514567,0.04761746,0.05044233,0.04838392,⋯,0.8047186,0.7281673,0.4420411,0.4745701,0.8036229,0.8446532,0.8059575,0.8769885,0.6174775,0.5569951
5,0.06426926,0.06406918,0.05702433,0.05276894,0.04848679,0.04424580,0.04301885,0.04311201,0.04557514,0.04929684,⋯,0.7491398,0.7379520,0.6675188,0.6551306,0.7406565,0.8998303,0.9689108,0.8887908,0.7388946,0.6410263
6,0.06286275,0.05956411,0.05707863,0.05208934,0.04850874,0.04746565,0.04748539,0.04717902,0.04801729,0.05065653,⋯,0.7746607,0.6766340,0.5459924,0.5157587,0.6739425,0.8702676,0.9066815,0.8294759,0.7525363,0.7513382


,X402.593_5nm,X407.593_5nm,X412.593_5nm,X417.593_5nm,X422.593_5nm,X427.593_5nm,X432.593_5nm,X437.593_5nm,X442.593_5nm,X447.593_5nm,⋯,X947.593_5nm,X952.593_5nm,X957.593_5nm,X962.593_5nm,X967.593_5nm,X972.593_5nm,X977.593_5nm,X982.593_5nm,X987.593_5nm,X992.593_5nm
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,0.04786391,0.04478547,0.04214955,0.04121219,0.04261843,0.04609276,0.05056049,0.05609938,0.06384817,0.07352870,⋯,0.2659656,0.2671230,0.2678679,0.2684674,0.2692454,0.2704128,0.2720405,0.2738601,0.2755660,0.2769153
2,0.04820802,0.04623885,0.04461647,0.04424838,0.04589012,0.04933364,0.05350729,0.05836299,0.06480323,0.07251967,⋯,0.2141355,0.2152685,0.2162140,0.2170847,0.2180113,0.2190688,0.2202912,0.2216226,0.2229784,0.2242359
3,0.05849977,0.05496151,0.05154779,0.04986053,0.05175844,0.05731791,0.06499972,0.07520754,0.09011172,0.10881299,⋯,0.3184974,0.3190839,0.3198678,0.3205158,0.3208263,0.3212036,0.3219709,0.3229237,0.3238387,0.3246974
4,0.06252788,0.05817240,0.05399310,0.05201525,0.05424306,0.06092052,0.07068518,0.08364273,0.10123249,0.12223676,⋯,0.3418603,0.3421727,0.3423974,0.3425062,0.3425074,0.3425434,0.3427278,0.3430264,0.3434254,0.3440410
5,0.05055413,0.04732800,0.04457956,0.04357744,0.04533789,0.04989059,0.05639766,0.06534486,0.07839488,0.09474391,⋯,0.2914092,0.2920809,0.2927031,0.2931446,0.2934322,0.2940614,0.2953721,0.2970353,0.2986498,0.2999403
6,0.06056797,0.05514715,0.05001802,0.04665174,0.04565842,0.04598097,0.04511881,0.04293868,0.04154127,0.04142498,⋯,0.4787045,0.4774562,0.4759385,0.4748448,0.4745815,0.4744645,0.4739159,0.4731926,0.4726754,0.4726835


In [12]:
img_raw_with_na <- cbind(img_resampled_bands, img_indices)
grd_raw_with_na <- cbind(grd_resampled_bands, grd_indices)
bg_raw_with_na <- cbind(bg_resampled_bands, bg_indices)

In [13]:
img_raw <- impute_spectra(img_raw_with_na)
grd_raw <- impute_spectra(inf_to_na(grd_raw_with_na))# note also dropping an Inf (liekly div by 0 in veg index)
bg_raw <- impute_spectra(bg_raw_with_na)

[1] "Imputing..."
[1] "Imputing..."
[1] "Imputing..."


In [14]:
write.csv(bg_raw, file="Data/gs/x_train/bison_gulch.csv")
write.csv(as.data.frame(bg_speclib$FncGrp1), file="Data/gs/y_train/bison_gulch.csv")

Apply the outlier transforms

In [15]:
grd_clipped <- clip_outliers(grd_raw)
grd_imputed <- impute_outliers_and_na(grd_raw)
grd_dropped <- grd_raw[detect_outliers_columnwise(grd_raw),]
img_clipped <- clip_outliers(img_raw)
img_imputed <- impute_outliers_and_na(img_raw)
img_dropped <- img_raw[detect_outliers_columnwise(img_raw),]

[1] "Imputing..."
[1] "Imputing..."


Now the center/scale transforms

In [ ]:
grd_raw_robust <- columnwise_robust_scale(grd_raw)
img_raw_robust <- columnwise_robust_scale(img_raw)
grd_raw_minmax <- columnwise_min_max_scale(grd_raw)
img_raw_minmax <- columnwise_min_max_scale(img_raw)
grd_raw_standard <- standardize_df(grd_raw)
img_raw_standard <- standardize_df(img_raw)

grd_clipped_robust <- columnwise_robust_scale(grd_clipped)
grd_imputed_robust <- columnwise_robust_scale(grd_imputed)
grd_dropped_robust <- columnwise_robust_scale(grd_dropped)
img_clipped_robust <- columnwise_robust_scale(img_clipped)
img_imputed_robust <- columnwise_robust_scale(img_imputed)
img_dropped_robust <- columnwise_robust_scale(img_dropped)

grd_clipped_minmax <- columnwise_min_max_scale(grd_clipped)
grd_imputed_minmax <- columnwise_min_max_scale(grd_imputed)
grd_dropped_minmax <- columnwise_min_max_scale(grd_dropped)
img_clipped_minmax <- columnwise_min_max_scale(img_clipped)
img_imputed_minmax <- columnwise_min_max_scale(img_imputed)
img_dropped_minmax <- columnwise_min_max_scale(img_dropped)

grd_clipped_standard <- standardize_df(grd_clipped)
grd_imputed_standard <- standardize_df(grd_imputed)
grd_dropped_standard <- standardize_df(grd_imputed)
img_clipped_standard <- standardize_df(img_clipped)
img_imputed_standard <- standardize_df(img_imputed)
img_dropped_standard <- standardize_df(img_dropped)


Now, let's save all these data to disk

In [ ]:
BASE_PATH <- "Data/gs/"
X_TRAIN_PATH <- paste0(BASE_PATH, "x_train/")
Y_TRAIN_PATH <- paste0(BASE_PATH, "y_train/")

X_TEST_PATH <- paste0(BASE_PATH, "x_test/")
Y_TEST_PATH <- paste0(BASE_PATH, "y_test/")

if(!dir.exists(BASE_PATH)){
    dir.create(BASE_PATH)
}
if(!dir.exists(X_TRAIN_PATH)){
    dir.create(X_TRAIN_PATH)
}
if(!dir.exists(Y_TRAIN_PATH)){
    dir.create(Y_TRAIN_PATH)
}
if(!dir.exists(X_TEST_PATH)){
    dir.create(X_TEST_PATH)
}
if(!dir.exists(Y_TEST_PATH)){
    dir.create(Y_TEST_PATH)
}


In [ ]:
write.csv(grd_clipped, file=paste0(X_TRAIN_PATH, "grd_clipped_raw.csv"))
write.csv(grd_clipped_minmax, file=paste0(X_TRAIN_PATH, "grd_clipped_minmax.csv"))
write.csv(grd_clipped_robust, file=paste0(X_TRAIN_PATH, "grd_clipped_robust.csv"))
write.csv(grd_clipped_standard, file=paste0(X_TRAIN_PATH, "grd_clipped_standard.csv"))

write.csv(grd_imputed, file=paste0(X_TRAIN_PATH, "grd_imputed_raw.csv"))
write.csv(grd_imputed_minmax, file=paste0(X_TRAIN_PATH, "grd_imputed_minmax.csv"))
write.csv(grd_imputed_robust, file=paste0(X_TRAIN_PATH, "grd_imputed_robust.csv"))
write.csv(grd_imputed_standard, file=paste0(X_TRAIN_PATH, "grd_imputed_standard.csv"))

write.csv(grd_dropped, file=paste0(X_TRAIN_PATH, "grd_dropped_raw.csv"))
write.csv(grd_dropped_minmax, file=paste0(X_TRAIN_PATH, "grd_dropped_minmax.csv"))
write.csv(grd_dropped_robust, file=paste0(X_TRAIN_PATH, "grd_dropped_robust.csv"))
write.csv(grd_dropped_standard, file=paste0(X_TRAIN_PATH, "grd_dropped_standard.csv"))

write.csv(grd_raw, file=paste0(X_TRAIN_PATH, "grd_raw_raw.csv"))
write.csv(grd_raw_minmax, file=paste0(X_TRAIN_PATH, "grd_raw_minmax.csv"))
write.csv(grd_raw_robust, file=paste0(X_TRAIN_PATH, "grd_raw_robust.csv"))
write.csv(grd_raw_standard, file=paste0(X_TRAIN_PATH, "grd_raw_standard.csv"))

In [ ]:
write.csv(grd_raw[,colnames(grd_indices)], file=paste0(X_TRAIN_PATH, "grd_indices_only.csv"))


## Labels for the above Data

In [ ]:
img_targets <- img_speclib$FncGrp1 %>% as.factor()
grd_targets <- grd_speclib$Functional_group1 %>% as.factor()

In [ ]:
write.csv(img_targets, file="Data/gs/y_train/img_indices_only.csv")
write.csv(grd_targets, file="Data/gs/y_train/grd_indices_only.csv")

In [ ]:
img_targets %>% table()

.
       Abiotic      Graminoid         Lichen           Moss     ShrubDecid 
           816            145             94             89            117 
ShrubEvergreen  TreeBroadleaf    TreeConifer 
           198            132           3091 

In [ ]:
grd_targets %>% table()

.
       Abiotic           Forb      Graminoid         Lichen           Moss 
            94            158            112            417            122 
    ShrubDecid ShrubEvergreen  TreeBroadleaf    TreeConifer 
           297            105             21             17 

In [ ]:
# drop entries with outliers to match training data
img_targets_dropped <- img_targets[detect_outliers_columnwise(img_raw)]
grd_targets_dropped <- grd_targets[detect_outliers_columnwise(grd_raw)]

In [ ]:
write.csv(grd_targets, file=paste0(Y_TRAIN_PATH, "grd_clipped_raw.csv"))
write.csv(grd_targets, file=paste0(Y_TRAIN_PATH, "grd_clipped_minmax.csv"))
write.csv(grd_targets, file=paste0(Y_TRAIN_PATH, "grd_clipped_robust.csv"))
write.csv(grd_targets, file=paste0(Y_TRAIN_PATH, "grd_clipped_standard.csv"))

write.csv(grd_targets, file=paste0(Y_TRAIN_PATH, "grd_imputed_raw.csv"))
write.csv(grd_targets, file=paste0(Y_TRAIN_PATH, "grd_imputed_minmax.csv"))
write.csv(grd_targets, file=paste0(Y_TRAIN_PATH, "grd_imputed_robust.csv"))
write.csv(grd_targets, file=paste0(Y_TRAIN_PATH, "grd_imputed_standard.csv"))

write.csv(grd_targets, file=paste0(Y_TRAIN_PATH, "grd_raw_raw.csv"))
write.csv(grd_targets, file=paste0(Y_TRAIN_PATH, "grd_raw_minmax.csv"))
write.csv(grd_targets, file=paste0(Y_TRAIN_PATH, "grd_raw_robust.csv"))
write.csv(grd_targets, file=paste0(Y_TRAIN_PATH, "grd_raw_standard.csv"))

write.csv(grd_targets_dropped, file=paste0(Y_TRAIN_PATH, "grd_dropped_raw.csv"))
write.csv(grd_targets_dropped, file=paste0(Y_TRAIN_PATH, "grd_dropped_minmax.csv"))
write.csv(grd_targets_dropped, file=paste0(Y_TRAIN_PATH, "grd_dropped_robust.csv"))
write.csv(grd_targets_dropped, file=paste0(Y_TRAIN_PATH, "grd_dropped_standard.csv"))

In [ ]:
write.csv(img_targets, file=paste0(Y_TRAIN_PATH, "img_clipped_raw.csv"))
write.csv(img_targets, file=paste0(Y_TRAIN_PATH, "img_clipped_minmax.csv"))
write.csv(img_targets, file=paste0(Y_TRAIN_PATH, "img_clipped_robust.csv"))
write.csv(img_targets, file=paste0(Y_TRAIN_PATH, "img_clipped_standard.csv"))

write.csv(img_imputed, file=paste0(Y_TRAIN_PATH, "img_imputed_raw.csv"))
write.csv(img_targets, file=paste0(Y_TRAIN_PATH, "img_imputed_minmax.csv"))
write.csv(img_targets, file=paste0(Y_TRAIN_PATH, "img_imputed_robust.csv"))
write.csv(img_targets, file=paste0(Y_TRAIN_PATH, "img_imputed_standard.csv"))

write.csv(img_targets, file=paste0(Y_TRAIN_PATH, "img_raw_raw.csv"))
write.csv(img_targets, file=paste0(Y_TRAIN_PATH, "img_raw_minmax.csv"))
write.csv(img_targets, file=paste0(Y_TRAIN_PATH, "img_raw_robust.csv"))
write.csv(img_targets, file=paste0(Y_TRAIN_PATH, "img_raw_standard.csv"))

write.csv(img_targets_dropped, file=paste0(Y_TRAIN_PATH, "img_dropped_raw.csv"))
write.csv(img_targets_dropped, file=paste0(Y_TRAIN_PATH, "img_dropped_minmax.csv"))
write.csv(img_targets_dropped, file=paste0(Y_TRAIN_PATH, "img_dropped_robust.csv"))
write.csv(img_targets_dropped, file=paste0(Y_TRAIN_PATH, "img_dropped_standard.csv"))

## Test Data

Build the test data, and save it with the same names as the training data

In [ ]:
set.seed(61718L)

permutation <-  permute::shuffle(length(img_targets))
sample <- create_stratified_sample(
    img_targets, 
    permutation = permutation,
    samples_per_pft = 15)
# split the data based on the above sample
img_targets_test <- img_targets[permutation][sample]
img_targets_train <- img_targets[permutation][-sample]
img_raw_test <- img_raw[permutation,][sample,]
img_raw_train <- img_raw[permutation,][-sample,]


[1] "Lichen"         "TreeConifer"    "TreeConifer"    "ShrubEvergreen"
[5] "TreeConifer"   


In [ ]:
img_targets_test %>% as.factor() %>% table()

.
       Abiotic      Graminoid         Lichen           Moss     ShrubDecid 
            15             15             15             15             15 
ShrubEvergreen  TreeBroadleaf    TreeConifer 
            15             15             15 

In [ ]:
# create the subsampled data and save them for each processing type/treatment

# clipped
img_clipped_train <- img_clipped[permutation,][-sample,]
img_clipped_test <- img_clipped[permutation,][sample,]
img_clipped_minmax_train <- img_clipped_minmax[permutation,][-sample,]
img_clipped_minmax_test <- img_clipped_minmax[permutation,][sample,]
img_clipped_robust_train <- img_clipped_robust[permutation,][-sample,]
img_clipped_robust_test <- img_clipped_robust[permutation,][sample,]
img_clipped_standard_train <- img_clipped_standard[permutation,][-sample]
img_clipped_standard_test <- img_clipped_standard[permutation,][sample,]

# raw (note one is done in the previous cell)
img_raw_minmax_train <- img_raw_minmax[permutation,][-sample,]
img_raw_minmax_test <- img_raw_minmax[permutation,][sample,]
img_raw_robust_train <- img_raw_robust[permutation,][-sample,]
img_raw_robust_test <- img_raw_robust[permutation,][sample,]
img_raw_standard_train <- img_raw_standard[permutation,][sample,]
img_raw_standard_test <- img_raw_standard[permutation,][sample,]

#imputed
img_imputed_train <- img_imputed[permutation,][-sample,]
img_imputed_test <- img_imputed[permutation,][sample,]
img_imputed_minmax_train <- img_imputed_minmax[permutation,][-sample,]
img_imputed_minmax_test <- img_imputed_minmax[permutation,][sample,]
img_imputed_robust_train <- img_imputed_robust[permutation,][-sample,]
img_imputed_robust_test <- img_imputed_robust[permutation,][sample,]
img_imputed_standard_train <- img_imputed_standard[permutation,][-sample,]
img_imputed_standard_test <- img_imputed_standard[permutation,][sample,]



In [ ]:
print(length(img_targets_test))
print(nrow(img_clipped_robust_test))

[1] 120
[1] 120


### Image-based Training Data

In [ ]:
write.csv(img_clipped_train, file=paste0(X_TRAIN_PATH, "img_clipped_raw.csv"))
write.csv(img_clipped_minmax_train, file=paste0(X_TRAIN_PATH, "img_clipped_minmax.csv"))
write.csv(img_clipped_robust_train, file=paste0(X_TRAIN_PATH, "img_clipped_robust.csv"))
write.csv(img_clipped_standard_train, file=paste0(X_TRAIN_PATH, "img_clipped_standard.csv"))

write.csv(img_imputed_train, file=paste0(X_TRAIN_PATH, "img_imputed_raw.csv"))
write.csv(img_imputed_minmax_train, file=paste0(X_TRAIN_PATH, "img_imputed_minmax.csv"))
write.csv(img_imputed_robust_train, file=paste0(X_TRAIN_PATH, "img_imputed_robust.csv"))
write.csv(img_imputed_standard_train, file=paste0(X_TRAIN_PATH, "img_imputed_standard.csv"))

write.csv(img_dropped, file=paste0(X_TRAIN_PATH, "img_dropped_raw.csv"))
write.csv(img_dropped_minmax, file=paste0(X_TRAIN_PATH, "img_dropped_minmax.csv"))
write.csv(img_dropped_robust, file=paste0(X_TRAIN_PATH, "img_dropped_robust.csv"))
write.csv(img_dropped_standard, file=paste0(X_TRAIN_PATH, "img_dropped_standard.csv"))

write.csv(img_raw_train, file=paste0(X_TRAIN_PATH, "img_raw_raw.csv"))
write.csv(img_raw_minmax_train, file=paste0(X_TRAIN_PATH, "img_raw_minmax.csv"))
write.csv(img_raw_robust_train, file=paste0(X_TRAIN_PATH, "img_raw_robust.csv"))
write.csv(img_raw_standard_train, file=paste0(X_TRAIN_PATH, "img_raw_standard.csv"))

In [ ]:
write.csv(img_targets_train, file=paste0(Y_TRAIN_PATH, "img_clipped_raw.csv"))
write.csv(img_targets_train, file=paste0(Y_TRAIN_PATH, "img_clipped_minmax.csv"))
write.csv(img_targets_train, file=paste0(Y_TRAIN_PATH, "img_clipped_robust.csv"))
write.csv(img_targets_train, file=paste0(Y_TRAIN_PATH, "img_clipped_standard.csv"))

write.csv(img_targets_train, file=paste0(Y_TRAIN_PATH, "img_imputed_raw.csv"))
write.csv(img_targets_train, file=paste0(Y_TRAIN_PATH, "img_imputed_minmax.csv"))
write.csv(img_targets_train, file=paste0(Y_TRAIN_PATH, "img_imputed_robust.csv"))
write.csv(img_targets_train, file=paste0(Y_TRAIN_PATH, "img_imputed_standard.csv"))

#write.csv(img_dropped, file=paste0(X_TRAIN_PATH, "img_dropped_raw.csv"))
#write.csv(img_dropped_minmax, file=paste0(X_TRAIN_PATH, "img_dropped_minmax.csv"))
#write.csv(img_dropped_robust, file=paste0(X_TRAIN_PATH, "img_dropped_robust.csv"))
#write.csv(img_dropped_standard, file=paste0(X_TRAIN_PATH, "img_dropped_standard.csv"))

write.csv(img_targets_train, file=paste0(Y_TRAIN_PATH, "img_raw_raw.csv"))
write.csv(img_targets_train, file=paste0(Y_TRAIN_PATH, "img_raw_minmax.csv"))
write.csv(img_targets_train, file=paste0(Y_TRAIN_PATH, "img_raw_robust.csv"))
write.csv(img_targets_train, file=paste0(Y_TRAIN_PATH, "img_raw_standard.csv"))

### Image Based Test Data
Note: this image-based test set is used for all the models (ground included)

In [ ]:
write.csv(img_clipped_test, file=paste0(X_TEST_PATH, "img_clipped_raw.csv"))
write.csv(img_clipped_minmax_test, file=paste0(X_TEST_PATH, "img_clipped_minmax.csv"))
write.csv(img_clipped_robust_test, file=paste0(X_TEST_PATH, "img_clipped_robust.csv"))
write.csv(img_clipped_standard_test, file=paste0(X_TEST_PATH, "img_clipped_standard.csv"))

write.csv(img_imputed_test, file=paste0(X_TEST_PATH, "img_imputed_raw.csv"))
write.csv(img_imputed_minmax_test, file=paste0(X_TEST_PATH, "img_imputed_minmax.csv"))
write.csv(img_imputed_robust_test, file=paste0(X_TEST_PATH, "img_imputed_robust.csv"))
write.csv(img_imputed_standard_test, file=paste0(X_TEST_PATH, "img_imputed_standard.csv"))

write.csv(img_dropped, file=paste0(X_TEST_PATH, "img_dropped_raw.csv"))
write.csv(img_dropped_minmax, file=paste0(X_TEST_PATH, "img_dropped_minmax.csv"))
write.csv(img_dropped_robust, file=paste0(X_TEST_PATH, "img_dropped_robust.csv"))
write.csv(img_dropped_standard, file=paste0(X_TEST_PATH, "img_dropped_standard.csv"))

write.csv(img_raw_test, file=paste0(X_TEST_PATH, "img_raw_raw.csv"))
write.csv(img_raw_minmax_test, file=paste0(X_TEST_PATH, "img_raw_minmax.csv"))
write.csv(img_raw_robust_test, file=paste0(X_TEST_PATH, "img_raw_robust.csv"))
write.csv(img_raw_standard_test, file=paste0(X_TEST_PATH, "img_raw_standard.csv"))

### Ground test (from the images)

In [ ]:
write.csv(img_clipped_test, file=paste0(X_TEST_PATH, "grd_clipped_raw.csv"))
write.csv(img_clipped_minmax_test, file=paste0(X_TEST_PATH, "grd_clipped_minmax.csv"))
write.csv(img_clipped_robust_test, file=paste0(X_TEST_PATH, "grd_clipped_robust.csv"))
write.csv(img_clipped_standard_test, file=paste0(X_TEST_PATH, "grd_clipped_standard.csv"))

write.csv(img_imputed_test, file=paste0(X_TEST_PATH, "grd_imputed_raw.csv"))
write.csv(img_imputed_minmax_test, file=paste0(X_TEST_PATH, "grd_imputed_minmax.csv"))
write.csv(img_imputed_robust_test, file=paste0(X_TEST_PATH, "grd_imputed_robust.csv"))
write.csv(img_imputed_standard_test, file=paste0(X_TEST_PATH, "grd_imputed_standard.csv"))

write.csv(img_dropped, file=paste0(X_TEST_PATH, "grd_dropped_raw.csv"))
write.csv(img_dropped_minmax, file=paste0(X_TEST_PATH, "grd_dropped_minmax.csv"))
write.csv(img_dropped_robust, file=paste0(X_TEST_PATH, "grd_dropped_robust.csv"))
write.csv(img_dropped_standard, file=paste0(X_TEST_PATH, "grd_dropped_standard.csv"))

write.csv(img_raw_test, file=paste0(X_TEST_PATH, "grd_raw_raw.csv"))
write.csv(img_raw_minmax_test, file=paste0(X_TEST_PATH, "grd_raw_minmax.csv"))
write.csv(img_raw_robust_test, file=paste0(X_TEST_PATH, "grd_raw_robust.csv"))
write.csv(img_raw_standard_test, file=paste0(X_TEST_PATH, "grd_raw_standard.csv"))

In [ ]:
write.csv(img_targets_test, file=paste0(Y_TEST_PATH, "grd_clipped_raw.csv"))
write.csv(img_targets_test, file=paste0(Y_TEST_PATH, "grd_clipped_minmax.csv"))
write.csv(img_targets_test, file=paste0(Y_TEST_PATH, "grd_clipped_robust.csv"))
write.csv(img_targets_test, file=paste0(Y_TEST_PATH, "grd_clipped_standard.csv"))

write.csv(img_targets_test, file=paste0(Y_TEST_PATH, "grd_imputed_raw.csv"))
write.csv(img_targets_test, file=paste0(Y_TEST_PATH, "grd_imputed_minmax.csv"))
write.csv(img_targets_test, file=paste0(Y_TEST_PATH, "grd_imputed_robust.csv"))
write.csv(img_targets_test, file=paste0(Y_TEST_PATH, "grd_imputed_standard.csv"))

write.csv(img_targets_test, file=paste0(Y_TEST_PATH, "grd_raw_raw.csv"))
write.csv(img_targets_test, file=paste0(Y_TEST_PATH, "grd_raw_minmax.csv"))
write.csv(img_targets_test, file=paste0(Y_TEST_PATH, "grd_raw_robust.csv"))
write.csv(img_targets_test, file=paste0(Y_TEST_PATH, "grd_raw_standard.csv"))

write.csv(grd_targets_dropped, file=paste0(Y_TEST_PATH, "grd_dropped_raw.csv"))
write.csv(grd_targets_dropped, file=paste0(Y_TEST_PATH, "grd_dropped_minmax.csv"))
write.csv(grd_targets_dropped, file=paste0(Y_TEST_PATH, "grd_dropped_robust.csv"))
write.csv(grd_targets_dropped, file=paste0(Y_TEST_PATH, "grd_dropped_standard.csv"))

In [ ]:
write.csv(img_targets_test, file=paste0(Y_TEST_PATH, "img_clipped_raw.csv"))
write.csv(img_targets_test, file=paste0(Y_TEST_PATH, "img_clipped_minmax.csv"))
write.csv(img_targets_test, file=paste0(Y_TEST_PATH, "img_clipped_robust.csv"))
write.csv(img_targets_test, file=paste0(Y_TEST_PATH, "img_clipped_standard.csv"))

write.csv(img_targets_test, file=paste0(Y_TEST_PATH, "img_imputed_raw.csv"))
write.csv(img_targets_test, file=paste0(Y_TEST_PATH, "img_imputed_minmax.csv"))
write.csv(img_targets_test, file=paste0(Y_TEST_PATH, "img_imputed_robust.csv"))
write.csv(img_targets_test, file=paste0(Y_TEST_PATH, "img_imputed_standard.csv"))

write.csv(img_targets_test, file=paste0(Y_TEST_PATH, "img_raw_raw.csv"))
write.csv(img_targets_test, file=paste0(Y_TEST_PATH, "img_raw_minmax.csv"))
write.csv(img_targets_test, file=paste0(Y_TEST_PATH, "img_raw_robust.csv"))
write.csv(img_targets_test, file=paste0(Y_TEST_PATH, "img_raw_standard.csv"))

write.csv(grd_targets_dropped, file=paste0(Y_TEST_PATH, "img_dropped_raw.csv"))
write.csv(grd_targets_dropped, file=paste0(Y_TEST_PATH, "img_dropped_minmax.csv"))
write.csv(grd_targets_dropped, file=paste0(Y_TEST_PATH, "img_dropped_robust.csv"))
write.csv(grd_targets_dropped, file=paste0(Y_TEST_PATH, "img_dropped_standard.csv"))

In [ ]:
bg_permutation <-  permute::shuffle(length(img_targets))
bg_sample <- create_stratified_sample(
    bg_targets, 
    permutation = permutation,
    samples_per_pft = 15)

In [ ]:
bg_targets_test <- bg_targets[permutation][sample]
bg_targets_train <- bg_targets[permutation][-sample]
bg_raw_test <- bg_raw[permutation,][sample,]
bg_raw_train <- bg_raw[permutation,][-sample,]

In [ ]:
# need to write the targets fror training
clip_transform <- create_clip_transform(
    img_raw
)

Warning message in fences[column_name] <- list(upper = upper_fence, lower = lower_fence):
“number of items to replace is not a multiple of replacement length”
Warning message in fences[column_name] <- list(upper = upper_fence, lower = lower_fence):
“number of items to replace is not a multiple of replacement length”
Warning message in fences[column_name] <- list(upper = upper_fence, lower = lower_fence):
“number of items to replace is not a multiple of replacement length”
Warning message in fences[column_name] <- list(upper = upper_fence, lower = lower_fence):
“number of items to replace is not a multiple of replacement length”
Warning message in fences[column_name] <- list(upper = upper_fence, lower = lower_fence):
“number of items to replace is not a multiple of replacement length”
Warning message in fences[column_name] <- list(upper = upper_fence, lower = lower_fence):
“number of items to replace is not a multiple of replacement length”
Warning message in fences[column_name] <- list

In [ ]:
save(clip_transform, file="./mle/clip_transform.rda")

In [ ]:
clipped_2 <- clip_transform(img_raw)# clipped 2

## Sensor Correction

In this section, we create the models (and do some data transforms) to make the sensor-correction models and create the corrected data (only three times).  

We do this first for the raw (including outliers) data.

In [ ]:
grd_resampled_to_match_img_bands <- resample_df(
    grd_bands,
    min_wavelength = 398,
    max_wavelength = 999,
    delta=1,
    drop_existing = TRUE
)
head(grd_resampled_to_match_img_bands)
head(img_bands)

,X398_5nm,X399_5nm,X400_5nm,X401_5nm,X402_5nm,X403_5nm,X404_5nm,X405_5nm,X406_5nm,X407_5nm,⋯,X990_5nm,X991_5nm,X992_5nm,X993_5nm,X994_5nm,X995_5nm,X996_5nm,X997_5nm,X998_5nm,X999_5nm
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,4.8846,4.8925,4.9184,4.9628,4.9102,4.7963,4.6673,4.5396,4.4311,4.3730,⋯,27.6272,27.6523,27.6757,27.6977,27.7181,27.7370,27.7544,27.7703,27.7846,27.7974
2,4.9516,4.9430,4.9209,4.8555,4.8264,4.7953,4.7073,4.6387,4.5811,4.5128,⋯,22.3594,22.3834,22.4069,22.4301,22.4529,22.4753,22.4974,22.5191,22.5404,22.5613
3,6.0398,6.0197,6.0054,5.9880,5.9352,5.8390,5.6852,5.5654,5.4625,5.3369,⋯,32.4257,32.4426,32.4593,32.4758,32.4922,32.5085,32.5246,32.5405,32.5563,32.5719
4,6.4706,6.4441,6.4293,6.4678,6.3939,6.2293,5.9987,5.8117,5.6694,5.5691,⋯,34.3678,34.3810,34.3951,34.4103,34.4264,34.4435,34.4616,34.4807,34.5008,34.5219
5,5.2403,5.2240,5.2112,5.1750,5.1250,5.0416,4.8941,4.7927,4.7200,4.6447,⋯,29.9326,29.9557,29.9775,29.9979,30.0170,30.0348,30.0513,30.0665,30.0803,30.0928
6,6.3948,6.3681,6.3422,6.3502,6.2172,5.9899,5.7240,5.5414,5.4176,5.3077,⋯,47.2582,47.2609,47.2660,47.2734,47.2832,47.2953,47.3097,47.3265,47.3457,47.3671


,X398,X399,X400,X401,X402,X403,X404,X405,X406,X407,⋯,X990,X991,X992,X993,X994,X995,X996,X997,X998,X999
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,0.05243007,0.04516057,0.03909831,0.03482924,0.03285911,0.03287741,0.03409723,0.03572572,0.03711334,0.03818382,⋯,0.5636831,0.5717859,0.5632404,0.5485100,0.5380677,0.5400187,0.5561121,0.5870420,0.6335022,0.6961869
2,0.03280578,0.03279741,0.03278975,0.03278255,0.03277556,0.03276862,0.03276165,0.03275454,0.03274727,0.03274001,⋯,0.4652565,0.4655240,0.4657567,0.4659595,0.4661377,0.4662962,0.4664396,0.4665724,0.4666992,0.4668245
3,0.02415163,0.02445285,0.02475288,0.02505101,0.02534659,0.02563954,0.02593011,0.02621855,0.02650515,0.02679029,⋯,0.4713047,0.4704058,0.4696060,0.4689031,0.4682946,0.4677750,0.4673299,0.4669434,0.4665996,0.4662826
4,0.03013168,0.03041985,0.03070853,0.03097938,0.03121466,0.03140237,0.03153387,0.03160058,0.03159647,0.03152571,⋯,0.4282925,0.4317823,0.4380752,0.4476607,0.4610279,0.4783732,0.4991071,0.5225098,0.5478615,0.5744424
5,0.02798701,0.02818876,0.02838857,0.02858507,0.02877691,0.02896310,0.02914283,0.02931529,0.02947982,0.02963626,⋯,0.4344142,0.4353318,0.4362373,0.4371232,0.4379823,0.4388103,0.4396119,0.4403929,0.4411592,0.4419167
6,0.01771090,0.02832461,0.03569997,0.04027814,0.04251483,0.04301360,0.04246431,0.04155789,0.04080908,0.04002789,⋯,0.6276267,0.6381394,0.6008782,0.5358806,0.4632139,0.3988209,0.3475733,0.3125049,0.2966494,0.3030405


In [ ]:
colnames(grd_resampled_to_match_img_bands) <- colnames(img_bands)

In [ ]:
grd_resampled_to_match_img_bands$targets <- grd_targets
img_bands_with_targets <- img_bands
img_bands_with_targets$targets <- img_targets

In [ ]:
matched_data <- create_matched_data(
    img_bands_with_targets,
    grd_resampled_to_match_img_bands,
    cols=c("targets","targets")# assumes joining on columns named "targets" in each data.frame
)
head(matched_data$left)
head(matched_data$right)

,X398,X399,X400,X401,X402,X403,X404,X405,X406,X407,⋯,X991,X992,X993,X994,X995,X996,X997,X998,X999,targets
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
3009,0.07348549,0.07412233,0.07475917,0.07539600,0.07603283,0.07666966,0.07730649,0.07794331,0.07858013,0.07921695,⋯,0.2527454,0.2528126,0.2528798,0.2529470,0.2530143,0.2530816,0.2531490,0.2532164,0.2532838,Abiotic
3049,0.07272919,0.07321966,0.07371138,0.07420552,0.07470318,0.07520499,0.07571131,0.07622251,0.07673873,0.07725930,⋯,0.1652452,0.1590823,0.1528258,0.1464928,0.1400994,0.1336582,0.1271812,0.1206803,0.1141676,Abiotic
2894,0.06993287,0.07047270,0.07101240,0.07155187,0.07209102,0.07262977,0.07316805,0.07370579,0.07424295,0.07477946,⋯,0.2136314,0.2131361,0.2126383,0.2121388,0.2116382,0.2111368,0.2106347,0.2101322,0.2096295,Abiotic
3150,0.07746122,0.07808746,0.07871372,0.07934002,0.07996637,0.08059279,0.08121931,0.08184594,0.08247273,0.08309968,⋯,0.2858592,0.2861733,0.2864878,0.2868024,0.2871173,0.2874322,0.2877473,0.2880624,0.2883775,Abiotic
2855,0.07336824,0.07387497,0.07438170,0.07488843,0.07539515,0.07590188,0.07640860,0.07691531,0.07742203,0.07792873,⋯,0.2687501,0.2689533,0.2691565,0.2693596,0.2695628,0.2697659,0.2699691,0.2701722,0.2703753,Abiotic
3201,0.07891200,0.07952668,0.08014135,0.08075602,0.08137069,0.08198536,0.08260002,0.08321469,0.08382934,0.08444399,⋯,0.2455605,0.2455618,0.2455632,0.2455646,0.2455660,0.2455673,0.2455687,0.2455700,0.2455714,Abiotic


,X398,X399,X400,X401,X402,X403,X404,X405,X406,X407,⋯,X991,X992,X993,X994,X995,X996,X997,X998,X999,targets
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
123,5.7498,5.7883,5.8291,5.8792,5.8807,5.8567,5.8380,5.8116,5.7855,5.7716,⋯,12.1430,12.1556,12.1681,12.1805,12.1927,12.2047,12.2166,12.2283,12.2398,Abiotic
829,30.7862,30.8971,31.0459,31.2268,31.3551,31.4182,31.3969,31.3692,31.3715,31.4733,⋯,47.9071,47.9409,47.9648,47.9788,47.9830,47.9773,47.9617,47.9363,47.9011,Abiotic
111,6.4009,6.3791,6.3679,6.4081,6.3689,6.2896,6.2227,6.1535,6.1086,6.1448,⋯,17.2888,17.3038,17.3191,17.3347,17.3505,17.3666,17.3830,17.3996,17.4165,Abiotic
371,5.4259,5.3604,5.3023,5.2732,5.1588,4.9765,4.7466,4.5334,4.3595,4.2659,⋯,47.2661,47.2729,47.2809,47.2902,47.3006,47.3122,47.3250,47.3390,47.3543,Abiotic
746,7.3284,7.3299,7.3275,7.3822,7.3813,7.3491,7.3198,7.2751,7.2211,7.1619,⋯,21.9503,21.9910,22.0346,22.0811,22.1305,22.1827,22.2379,22.2960,22.3569,Abiotic
1140,6.4009,6.3791,6.3679,6.4081,6.3689,6.2896,6.2227,6.1535,6.1086,6.1448,⋯,17.2888,17.3038,17.3191,17.3347,17.3505,17.3666,17.3830,17.3996,17.4165,Abiotic


In [ ]:
correction_model <- build_columnwise_sensor_correction_model(
    matched_data$left,
    matched_data$right,
    grouping_variables =c("targets","targets")
)


Call:
lm(formula = left_vec ~ right_vec)

Residuals:
      Min        1Q    Median        3Q       Max 
-0.038065 -0.008574 -0.005422  0.005334  0.048681 

Coefficients:
             Estimate Std. Error t value Pr(>|t|)    
(Intercept) 0.0196877  0.0007257   27.13   <2e-16 ***
right_vec   0.0014906  0.0000556   26.81   <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.01427 on 647 degrees of freedom
Multiple R-squared:  0.5263,	Adjusted R-squared:  0.5256 
F-statistic: 718.8 on 1 and 647 DF,  p-value: < 2.2e-16


Call:
lm(formula = left_vec ~ right_vec)

Residuals:
      Min        1Q    Median        3Q       Max 
-0.038752 -0.008593 -0.005612  0.005138  0.038505 

Coefficients:
             Estimate Std. Error t value Pr(>|t|)    
(Intercept) 1.994e-02  7.107e-04   28.06   <2e-16 ***
right_vec   1.470e-03  5.445e-05   26.99   <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error:

In [ ]:
print(correction_model)

$X398

Call:
lm(formula = left_vec ~ right_vec)

Coefficients:
(Intercept)    right_vec  
   0.019688     0.001491  


$X399

Call:
lm(formula = left_vec ~ right_vec)

Coefficients:
(Intercept)    right_vec  
    0.01994      0.00147  


$X400

Call:
lm(formula = left_vec ~ right_vec)

Coefficients:
(Intercept)    right_vec  
   0.020175     0.001458  


$X401

Call:
lm(formula = left_vec ~ right_vec)

Coefficients:
(Intercept)    right_vec  
   0.020345     0.001456  


$X402

Call:
lm(formula = left_vec ~ right_vec)

Coefficients:
(Intercept)    right_vec  
   0.020561     0.001461  


$X403

Call:
lm(formula = left_vec ~ right_vec)

Coefficients:
(Intercept)    right_vec  
   0.020841     0.001464  


$X404

Call:
lm(formula = left_vec ~ right_vec)

Coefficients:
(Intercept)    right_vec  
   0.021151     0.001468  


$X405

Call:
lm(formula = left_vec ~ right_vec)

Coefficients:
(Intercept)    right_vec  
   0.021435     0.001472  


$X406

Call:
lm(formula = left_vec ~ right_vec)


In [ ]:
grd_corrected_bands <- apply_sensor_correction_model(
    correction_model,
    grd_resampled_to_match_img_bands,
    ignore_cols=c("targets")
)
head(grd_corrected_bands)

[1] "Correcting X398"
[1] "Correcting X399"
[1] "Correcting X400"
[1] "Correcting X401"
[1] "Correcting X402"
[1] "Correcting X403"
[1] "Correcting X404"
[1] "Correcting X405"
[1] "Correcting X406"
[1] "Correcting X407"
[1] "Correcting X408"
[1] "Correcting X409"
[1] "Correcting X410"
[1] "Correcting X411"
[1] "Correcting X412"
[1] "Correcting X413"
[1] "Correcting X414"
[1] "Correcting X415"
[1] "Correcting X416"
[1] "Correcting X417"
[1] "Correcting X418"
[1] "Correcting X419"
[1] "Correcting X420"
[1] "Correcting X421"
[1] "Correcting X422"
[1] "Correcting X423"
[1] "Correcting X424"
[1] "Correcting X425"
[1] "Correcting X426"
[1] "Correcting X427"
[1] "Correcting X428"
[1] "Correcting X429"
[1] "Correcting X430"
[1] "Correcting X431"
[1] "Correcting X432"
[1] "Correcting X433"
[1] "Correcting X434"
[1] "Correcting X435"
[1] "Correcting X436"
[1] "Correcting X437"
[1] "Correcting X438"
[1] "Correcting X439"
[1] "Correcting X440"
[1] "Correcting X441"
[1] "Correcting X442"
[1] "Corre

,X398,X399,X400,X401,X402,X403,X404,X405,X406,X407,⋯,X991,X992,X993,X994,X995,X996,X997,X998,X999,targets
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
1,0.02696849,0.02712975,0.02734496,0.02757254,0.02773292,0.02786395,0.02800241,0.02811774,0.02822418,0.02833745,⋯,0.2013615,0.1981552,0.1938248,0.1903206,0.1878039,0.1839859,0.1789929,0.1741364,0.1682048,Lichen
2,0.02706835,0.02720396,0.02734861,0.02741628,0.02761052,0.02786249,0.02806113,0.02826362,0.02844536,0.02854388,⋯,0.1821710,0.1790505,0.1744729,0.1708238,0.1682382,0.1637371,0.1574840,0.1511437,0.1431531,Lichen
3,0.02869039,0.02878618,0.02892960,0.02906553,0.02922998,0.02939076,0.02949665,0.02962777,0.02974497,0.02976074,⋯,0.2188089,0.2155005,0.2113784,0.2079988,0.2055467,0.2023598,0.1985315,0.1950576,0.1910479,Lichen
4,0.02933253,0.02940984,0.02954756,0.02976425,0.02989993,0.02996227,0.02995686,0.02999034,0.03005004,0.03010361,⋯,0.2258690,0.2225197,0.2184853,0.2151611,0.2127420,0.2098207,0.2064786,0.2035831,0.2003775,Lichen
5,0.02749868,0.02761689,0.02777181,0.02788157,0.02804664,0.02822314,0.02833534,0.02849032,0.02865016,0.02873864,⋯,0.2097510,0.2065015,0.2022752,0.1988333,0.1963482,0.1928331,0.1883981,0.1842017,0.1791869,Lichen
6,0.02921954,0.02929816,0.02942059,0.02959299,0.02964185,0.02961172,0.02955361,0.02959244,0.02967877,0.02971763,⋯,0.2727805,0.2691897,0.2657414,0.2627692,0.2605314,0.2593090,0.2590948,0.2599005,0.2618342,ShrubDecid


In [ ]:
grd_corrected_indices <- get_vegetation_indices(grd_corrected_bands, NULL)
head(grd_corrected_indices)

,Boochs,Boochs2,CARI,Carter,Carter2,Carter3,Carter4,Carter5,Carter6,CI,⋯,TCARI,TCARIOSAVI,TCARI2,TCARI2OSAVI2,TGI,TVI,Vogelmann,Vogelmann2,Vogelmann3,Vogelmann4
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,0.002188828,0.001846240,0.2719767,3.755394,0.5511474,0.4211933,0.7026950,1.247888,0.07923122,1.010749,⋯,0.05881130,0.1923059,0.09237473,0.5852788,1.376441,6.048321,1.146709,-0.03322667,1.0222095,-0.03421461
2,0.002399323,0.001733916,0.2554836,3.315345,0.5320999,0.4137054,0.7090959,1.290439,0.07519772,1.014324,⋯,0.06637860,0.2104420,0.08893242,0.5692666,1.553611,6.270070,1.135502,-0.02884602,0.9407531,-0.02968302
3,0.002101335,0.002096861,0.2942251,4.073443,0.5623409,0.4556386,0.6929105,1.183224,0.09317601,1.007811,⋯,0.05085589,0.1746707,0.10958257,0.6609402,1.366523,6.187989,1.165590,-0.03813877,1.1331657,-0.03937819
4,0.002092272,0.002164736,0.2987104,4.127738,0.5586151,0.4567733,0.6848727,1.170913,0.09671020,1.005619,⋯,0.04928632,0.1692077,0.11564068,0.6819879,1.375553,6.308593,1.170996,-0.03971264,1.1795753,-0.04103479
5,0.002189149,0.002037904,0.2837821,3.986724,0.5530718,0.4443074,0.6943873,1.197440,0.08834606,1.006919,⋯,0.05315722,0.1796784,0.10474194,0.6369555,1.393422,6.174849,1.159186,-0.03601401,1.0805045,-0.03716122
6,0.004159070,0.004119772,0.2791441,2.755524,0.2976833,0.2178025,0.5056777,1.567244,0.07148862,1.012350,⋯,0.10824120,0.1998117,0.18217696,0.5776702,2.733532,13.806734,1.361022,-0.08144549,1.0109666,-0.08695752


In [ ]:
grd_corrected_resampled_bands <- resample_df(grd_corrected_bands, drop_existing=TRUE)
head(grd_corrected_resampled_bands)

Using spline to predict value at new bands...

Beware the spectra are now partially smoothed.



,X402.593_5nm,X407.593_5nm,X412.593_5nm,X417.593_5nm,X422.593_5nm,X427.593_5nm,X432.593_5nm,X437.593_5nm,X442.593_5nm,X447.593_5nm,⋯,X947.593_5nm,X952.593_5nm,X957.593_5nm,X962.593_5nm,X967.593_5nm,X972.593_5nm,X977.593_5nm,X982.593_5nm,X987.593_5nm,X992.593_5nm
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,0.02782071,0.02838161,0.02881200,0.02953802,0.03041885,0.03140579,0.03268065,0.03426034,0.03629715,0.03846875,⋯,0.1986401,0.1973474,0.1981889,0.1948016,0.2030131,0.2054632,0.2019880,0.2064505,0.2034961,0.1959680
2,0.02774295,0.02861989,0.02920462,0.03005340,0.03088526,0.03192904,0.03320195,0.03467239,0.03640983,0.03831450,⋯,0.1823018,0.1811807,0.1820199,0.1776538,0.1865792,0.1881836,0.1836699,0.1874669,0.1826401,0.1768311
3,0.02932662,0.02981599,0.03036616,0.03084692,0.03184513,0.03324964,0.03511629,0.03756044,0.04078956,0.04457036,⋯,0.2152218,0.2135408,0.2143877,0.2122952,0.2195161,0.2225082,0.2197304,0.2242552,0.2226292,0.2133345
4,0.02993976,0.03019161,0.03080884,0.03124855,0.03212492,0.03385584,0.03612365,0.03896951,0.04269738,0.04691695,⋯,0.2225767,0.2206195,0.2216415,0.2195323,0.2264225,0.2297559,0.2270746,0.2315493,0.2303837,0.2203612
5,0.02813808,0.02876623,0.02918112,0.02998573,0.03074075,0.03208510,0.03361788,0.03593619,0.03872062,0.04215217,⋯,0.2067125,0.2050831,0.2059476,0.2030758,0.2107600,0.2134579,0.2101491,0.2149503,0.2126399,0.2043252
6,0.02962056,0.02976571,0.03015992,0.03043716,0.03079569,0.03134480,0.03185086,0.03207111,0.03235061,0.03283675,⋯,0.2657699,0.2627438,0.2633904,0.2638961,0.2685468,0.2743417,0.2734250,0.2788993,0.2815916,0.2670936


In [ ]:
write.csv(
    cbind(grd_corrected_resampled_bands, grd_corrected_indices), 
    file=paste0(X_TRAIN_PATH, "grd_raw_corrected.csv")
    )

# save labels also
write.csv(
    grd_resampled_to_match_img_bands$targets,
    file=paste0(Y_TRAIN_PATH, "grd_raw_corrected.csv"))

Now that that is done, we will move on to the clipped data

In [ ]:
grd_resampled_to_img_clipped <- resample_df(
    clip_outliers(grd_bands),
    min_wavelength = 398,
    max_wavelength = 999,
    delta=1,
    drop_existing = TRUE
)

colnames(grd_resampled_to_img_clipped) <- colnames(img_bands)

grd_resampled_to_img_clipped$targets <- grd_targets
img_bands_with_targets <- img_bands
img_bands_with_targets$targets <- img_targets

matched_data_clipped <- create_matched_data(
    img_bands_with_targets,
    grd_resampled_to_img_clipped,
    cols=c("targets","targets")# assumes joining on columns named "targets" in each data.frame
)

correction_model <- build_columnwise_sensor_correction_model(
    matched_data_clipped$left,
    matched_data_clipped$right
)
grd_corrected_clipped_bands <- apply_sensor_correction_model(
    correction_model,
    grd_resampled_to_match_img_bands,
    ignore_cols=c("targets")
)
grd_corrected_clipped_indices <- get_vegetation_indices(grd_corrected_bands, NULL)
grd_corrected_clipped_resampled_bands <- resample_df(grd_corrected_bands, drop_existing=TRUE)





Call:
lm(formula = left_vec ~ right_vec)

Residuals:
      Min        1Q    Median        3Q       Max 
-0.039059 -0.012959 -0.003337  0.010969  0.064740 

Coefficients:
             Estimate Std. Error t value Pr(>|t|)    
(Intercept) 0.0176396  0.0013061   13.51   <2e-16 ***
right_vec   0.0021383  0.0001626   13.15   <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.0182 on 647 degrees of freedom
Multiple R-squared:  0.211,	Adjusted R-squared:  0.2098 
F-statistic:   173 on 1 and 647 DF,  p-value: < 2.2e-16


Call:
lm(formula = left_vec ~ right_vec)

Residuals:
      Min        1Q    Median        3Q       Max 
-0.038734 -0.012776 -0.003231  0.010307  0.061052 

Coefficients:
             Estimate Std. Error t value Pr(>|t|)    
(Intercept) 0.0176101  0.0012781   13.78   <2e-16 ***
right_vec   0.0021688  0.0001598   13.57   <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0

Using spline to predict value at new bands...

Beware the spectra are now partially smoothed.



In [ ]:
write.csv(
    cbind(
        grd_corrected_clipped_indices,
        grd_corrected_clipped_resampled_bands
    ),
    file=paste0(
        X_TRAIN_PATH,
        "grd_clipped_corrected.csv"
    )
)

# save labels also
write.csv(
    grd_resampled_to_match_img_bands$targets,
    file=paste0(Y_TRAIN_PATH, "grd_clipped_corrected.csv"))

And finally the dropped outlier one

notes for later: should probably try PCA here.  clip -> scale -> PCA -> subset (and scale again for models like SVM and kNN)

...to be continued